<a href="https://colab.research.google.com/github/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics/blob/master/hw03/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! rm -rf Machine-Learning-in-Atmospheric-Thermodynamics
! git clone https://github.com/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics.git



Cloning into 'Machine-Learning-in-Atmospheric-Thermodynamics'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 318 (delta 99), reused 189 (delta 43), pack-reused 32
Receiving objects: 100% (318/318), 178.70 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [72]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split

path='Machine-Learning-in-Atmospheric-Thermodynamics/hw03'
df=pd.read_csv(path+'/inversion_LTS.47918.csv')
df.head()

x=df.iloc[:, 1].astype(np.float32)
y=df.iloc[:, -1].astype(np.float32)

#print(x[0:5])
#print(y[0:5])

x_temp,x_valid,y_temp,y_valid=train_test_split(x,y,test_size=0.2,random_state=1)
x_train,x_test,y_train,y_test=train_test_split(x_temp,y_temp,test_size=0.2,random_state=1)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train.values.reshape(-1,1))
x_test=scaler.transform(x_test.values.reshape(-1,1))
x_valid=scaler.transform(x_valid.values.reshape(-1,1))
y_train=y_train.values.reshape(-1,1)
y_test=y_test.values.reshape(-1,1)
y_valid=y_valid.values.reshape(-1,1)

print('finished')

finished


In [73]:
import torch
from torch.utils.data import Dataset, DataLoader

class TrainData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.x_data)

train_data=TrainData(torch.FloatTensor(x_train),torch.FloatTensor(y_train))
test_data=TrainData(torch.FloatTensor(x_test),torch.FloatTensor(y_test))

class ValidData(Dataset):
    
    def __init__(self, x_data):
        self.x_data = x_data
        
    def __getitem__(self, index):
        return self.x_data[index]
        
    def __len__ (self):
        return len(self.x_data)
    
valid_data=ValidData(torch.FloatTensor(x_valid))

BATCH_SIZE=64
train_loader=DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True)
valid_loader=DataLoader(dataset=valid_data,batch_size=1)

print('finished')

finished


In [81]:
from torch import nn
from torchvision.transforms import ToTensor, Lambda, Compose
import time

class BinaryClassification(nn.Module):
  def __init__(self):
    super(BinaryClassification, self).__init__()
    self.BC = nn.Sequential(
      nn.Linear(1, 128),   
      nn.ReLU(),     
      nn.BatchNorm1d(128),   
      nn.Linear(128, 128),
      nn.ReLU(),
      nn.BatchNorm1d(128),
      nn.Dropout(p=0.1),
      nn.Linear(128, 1),
      nn.Sigmoid()
    )

  def forward(self, x):
    freq = self.BC(x)
    binary = torch.round(freq)
    return binary

print('defined model')

defined model


In [88]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = BinaryClassification().to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.train()
    train_loss=0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)  
        loss = loss_fn(pred, y) 
        train_loss += loss.item()

        optimizer.zero_grad()  
        loss.backward()      
        optimizer.step()      
       
        #if batch % 100 == 0:
        #    loss, current = loss.item(), batch * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    return train_loss

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad(): 
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred==y).sum().float()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return test_loss

# training start
epochs=1000

tt=time.time()
train_loss=[]
test_loss=[]
for t in range(epochs):
    print(f"Epoch {t+1}-------------------------------")
    loss = train(train_loader, model, loss_fn, optimizer)
    train_loss.append(loss)
    loss = test(test_loader, model, loss_fn)
    test_loss.append(loss)

elapse=time.time()-tt
print(elapse)

Epoch 1-------------------------------
Accuracy: 60.2%, Avg loss: 40.087426
Epoch 2-------------------------------
Accuracy: 60.8%, Avg loss: 39.393601
Epoch 3-------------------------------
Accuracy: 59.6%, Avg loss: 40.511533
Epoch 4-------------------------------
Accuracy: 62.0%, Avg loss: 37.832961
Epoch 5-------------------------------
Accuracy: 61.6%, Avg loss: 38.396577
Epoch 6-------------------------------
Accuracy: 61.1%, Avg loss: 38.798363
Epoch 7-------------------------------
Accuracy: 58.7%, Avg loss: 41.465774
Epoch 8-------------------------------
Accuracy: 59.1%, Avg loss: 40.913318
Epoch 9-------------------------------
Accuracy: 61.6%, Avg loss: 38.504464
Epoch 10-------------------------------
Accuracy: 57.9%, Avg loss: 41.977307
Epoch 11-------------------------------
Accuracy: 57.7%, Avg loss: 42.183780
Epoch 12-------------------------------
Accuracy: 57.9%, Avg loss: 41.815476
Epoch 13-------------------------------
Accuracy: 62.0%, Avg loss: 37.832961
Epoch 14

KeyboardInterrupt: ignored